<a href="https://colab.research.google.com/github/DatNguyen2084/DLDH-Metaphor-detection/blob/main/BERT_Metaphor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DLDH Seminar - Metaphor Dectection

In [ ]:
!pip install -q sentence_transformers
!pip install -q datasets

     |████████████████████████████████| 78 kB 3.4 MB/s 
     |████████████████████████████████| 3.4 MB 11.8 MB/s 
     |████████████████████████████████| 3.3 MB 19.6 MB/s 
     |████████████████████████████████| 1.2 MB 33.4 MB/s 
     |████████████████████████████████| 61 kB 470 kB/s 
     |████████████████████████████████| 895 kB 34.2 MB/s 
     |████████████████████████████████| 596 kB 40.6 MB/s 
     |████████████████████████████████| 298 kB 5.4 MB/s 
     |████████████████████████████████| 243 kB 40.1 MB/s 
     |████████████████████████████████| 132 kB 45.6 MB/s 
     |████████████████████████████████| 1.1 MB 26.5 MB/s 
     |████████████████████████████████| 271 kB 39.7 MB/s 
     |████████████████████████████████| 160 kB 45.7 MB/s 
     |████████████████████████████████| 192 kB 45.7 MB/s 


In [ ]:
import pandas as pd

In [ ]:
# Mount Google Drive
# The following data is needed: https://drive.google.com/drive/folders/1uPnLexQh8kbV5ErVR7ksagVKP_wDd4a0?usp=sharing
# Create a shortcut to your Drive ("Drive-Verknüpfung hinzufügen" zu "Meine Ablage")
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT_PATH = '/content/drive/My Drive/DLDH'
DATA_PATH = '/data'
MODEL_PATH = '/model'

## Utils

In [ ]:
# by calling this method, the NaN values in the column 'Stärkegrad (A, B, C)' get replaced by the most used value in the column
def set_default_staerkegrad_df(df):
  if ('Stärkegrad (A, B, C)' not in df.columns):
    print('[set_default_staerkegrad_df]: Given dataframe does not consist of a column "Staerkegrad"!')
    return df
  else:
    most_used_staerkegrad = df['Stärkegrad (A, B, C)'].value_counts().index[0]
    df['Stärkegrad (A, B, C)'].fillna(value=most_used_staerkegrad, inplace=True)
    print(df['Stärkegrad (A, B, C)'])
    return df

### Load Annotated Data (positive and negative)

In [ ]:
raw_df = pd.read_csv(ROOT_PATH + DATA_PATH + '/Annotationen-Stufe-2.csv')
# Drow rows where Textstelle is NaN
index_names = raw_df[raw_df['Textstelle'].isnull()].index
raw_df.drop(index_names, inplace=True)
#raw_df = set_default_staerkegrad_df(raw_df) 
raw_df

,Unnamed: 0,Seite,Textstelle,Metapher?,Fokus,Rahmen,"Stärkegrad (A, B, C)",Begründung/Kommentar,Annotator,Unnamed: 2,Semantikerweiterung?,Unersetzlich?,sprachlich irregulär?,pointiert?
0,0,NaN,Bei Beobachtung solchen moralischen Wertes ka...,Metapher,das Horoskop stellen,einer Nation,B,Horoskop stellen - bezogen auf Nationen ist da...,B,NaN,NaN,NaN,NaN,NaN
1,1,NaN,Die Zellen verschmelzen miteinander.,Nein,NaN,NaN,NaN,Fachausdruck,B,NaN,NaN,NaN,NaN,NaN
2,2,NaN,"Diese wolle die bittere Auslese, ohne die auc...",Metapher,bittere,Auslese,A,"Unauffällig, aber doch metaphorisch: Dass eine...",B,NaN,NaN,NaN,NaN,NaN
3,3,NaN,"Wenn es dem Verfasser gelungen ist, ein gesic...",Metapher,ein gesichertes Fundament und die ersten Pfeil...,die Lösung der hier zur Bearbeitung gestellten...,A,"Bruch, Fokus nicht ohne Bedeutungsverlust erse...",B,NaN,NaN,NaN,NaN,NaN
4,4,NaN,In Californien ist ebenso die früher dort hei...,Nein,NaN,NaN,NaN,"kein Bruch, nur auffälliger Ausdruck, keine Be...",B,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2579,2579,NaN,In ähnlicher Weise sollen noch mehrere andere ...,Nein,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN
2580,2580,NaN,"Die Amerikaner folgern, da sie alle Rohmateria...",Metapher,einer turmhohen Mauer,Schutzzöllen,A,NaN,B,NaN,NaN,NaN,NaN,NaN
2581,2581,NaN,Sie könne bei ihrer jetzigen großen Kraft und ...,Metapher,mütterlichen Päppelflasche,Sie,B,NaN,B,NaN,NaN,NaN,NaN,NaN
2582,2582,NaN,"Schwarze Flecken von Habgier, Raubsucht, Unger...",Nein,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN


### Get data by annotator

In [ ]:
# {'T', 'A', 'P', 'B', 'K'}
import numpy as np
def get_isMetaphor_data_by_annotator(annotator = 'B')-> pd.DataFrame:
  if annotator == 'all':
    return raw_df.loc[(raw_df['Annotator'].isin(['T', 'A', 'P', 'B', 'K']) ) & (raw_df['Metapher?'] == 'Metapher') & (raw_df['Fokus'] == 'bittere')]
  annotators = set(raw_df['Annotator'].tolist())
  if annotator not in annotators:
    raise Exception('The given Annotator is not found')
  return raw_df.loc[(raw_df['Annotator'] == annotator) & (raw_df['Metapher?'] == 'Metapher')]

metaphor_all = get_isMetaphor_data_by_annotator('all')
metaphor_all

,Unnamed: 0,Seite,Textstelle,Metapher?,Fokus,Rahmen,"Stärkegrad (A, B, C)",Begründung/Kommentar,Annotator,Unnamed: 2,Semantikerweiterung?,Unersetzlich?,sprachlich irregulär?,pointiert?
2,2,NaN,"Diese wolle die bittere Auslese, ohne die auc...",Metapher,bittere,Auslese,A,"Unauffällig, aber doch metaphorisch: Dass eine...",B,NaN,NaN,NaN,NaN,NaN
469,469,NaN,"Diese wolle die bittere Auslese, ohne die auc...",Metapher,bittere,Auslese,B,"Unauffällig, aber doch metaphorisch: Dass eine...",A,NaN,NaN,NaN,NaN,NaN
580,580,NaN,"Diese wolle die bittere Auslese, ohne die auc...",Metapher,bittere,Auslese,A,"Unauffällig, aber doch metaphorisch: Dass eine...",P,NaN,NaN,NaN,NaN,NaN
1609,1609,NaN,"Diese wolle die bittere Auslese, ohne die auc...",Metapher,bittere,Auslese,A,"Unauffällig, aber doch metaphorisch: Dass eine...",K,NaN,NaN,NaN,NaN,NaN


In [ ]:
len(metaphor_all)

53

In [ ]:
def get_notMetaphor_data_by_annotator(annotator='B', size=0):
  if annotator == 'all':
    return raw_df.loc[(raw_df['Annotator'].isin(['T', 'A', 'P', 'B', 'K']) ) & (raw_df['Metapher?'] == 'Nein')].head(size)
  return raw_df.loc[(raw_df['Annotator'] == annotator) & (raw_df['Metapher?'] == 'Nein')].head(size)

not_metaphor_B = get_notMetaphor_data_by_annotator('K', 9999)
not_metaphor_B

,Unnamed: 0,Seite,Textstelle,Metapher?,Fokus,Rahmen,"Stärkegrad (A, B, C)",Begründung/Kommentar,Annotator,Unnamed: 2,Semantikerweiterung?,Unersetzlich?,sprachlich irregulär?,pointiert?
853,853,NaN,Erst durch die Verschmelzung der Descendenzth...,Nein,NaN,NaN,NaN,Ersetzbar (Verschmelzung -> Zusammenführung),K,NaN,NaN,NaN,NaN,NaN
858,858,NaN,Durch die reichen Forschungsergebnisse der le...,Nein,NaN,NaN,NaN,erneut diese Licht-Dunkel-Thematik. Würde es m...,K,NaN,NaN,NaN,NaN,NaN
859,859,NaN,Das wurde in unzweifelhafter Weise zuerst 188...,Nein,NaN,NaN,NaN,"""weitere Entdeckungen auf dem Gebiete"" fällt z...",K,NaN,NaN,NaN,NaN,NaN
860,860,NaN,Im Frühjahr 1875 machte Oskar Hertwig unter H...,Nein,NaN,NaN,NaN,"""Wendepunkt"" = Floskel",K,NaN,NaN,NaN,NaN,NaN
861,861,NaN,Natürlich war dieses wichtige Forschungsgebie...,Nein,NaN,NaN,NaN,"""emsig"" -> unermüdlich",K,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,2466,NaN,"177\tUnter diesen byzantischen Unsitten, die s...",Nein,NaN,NaN,NaN,NaN,K,NaN,NaN,NaN,NaN,NaN
2467,2467,NaN,"185\tDie wichtige Theorie von der ""spezifische...",Nein,NaN,NaN,NaN,NaN,K,NaN,NaN,NaN,NaN,NaN
2468,2468,NaN,187\tSo pflanzt sich das nämliche geistige Ban...,Nein,NaN,NaN,NaN,NaN,K,NaN,NaN,NaN,NaN,NaN
2470,2470,NaN,194\tErst das rasche Emporblühen der Naturwiss...,Nein,NaN,NaN,NaN,NaN,K,NaN,NaN,NaN,NaN,NaN


### Load pre-trained BERT

In [ ]:
from transformers import BertForSequenceClassification, AutoTokenizer
model = BertForSequenceClassification.from_pretrained(ROOT_PATH + MODEL_PATH, cache_dir=None, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("redewiedergabe/bert-base-historical-german-rw-cased")

Some weights of the model checkpoint at /content/drive/My Drive/DLDH/model were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification 

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

 Train Model